In [1]:
# This is file for train, prediction

import pandas as pd

dirpath = 'C:/Users/rihot/Desktop/Deep_learning/DACON_used_car_price/'

train = pd.read_csv('data/preprocessed_train.csv')
test = pd.read_csv('data/preprocessed_test.csv')

In [2]:
Y = train[ ['target'] ].values
X = train[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values

X_test = test[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values

In [3]:
X.shape, Y.shape

((1015, 10), (1015, 1))

In [4]:
from sklearn.preprocessing import MinMaxScaler

scalerX = MinMaxScaler()
scalerX.fit(X)
X = scalerX.transform(X)
X_test = scalerX.transform(X_test)

scalerY = MinMaxScaler()
scalerY.fit(Y)
Y = scalerY.transform(Y)

In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import optuna
from optuna import Trial
from lightgbm import LGBMRegressor
import xgboost as xgb

def LGBM(trial : Trial, X, Y, test):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'hist',
        'predictor': 'cpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = LGBMRegressor(**param)
    LGBM_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(LGBM_model.predict(X_test), y_test, squared=False)

    return score

In [11]:
from optuna.samplers import TPESampler

study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : LGBM(trial, X,  Y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2022-06-02 15:31:43,348] A new study created in memory with name: no-name-60ddd04b-bba5-427d-b02f-c778a9be0aac
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\basic.py:1487: UserWarning: predictor keyword has been found in `params` and will be ignored.
Please use predictor argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} key

[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.001108756212651572. Current value: lambda_l2=0.001108756212651572


[I 2022-06-02 15:31:44,833] Trial 0 finished with value: 0.05127271337130929 and parameters: {'n_estimators': 1292, 'max_depth': 8, 'colsample_bytree': 0.6, 'lambda': 0.001108756212651572, 'alpha': 0.003128240936599907, 'subsample': 1.0}. Best is trial 0 with value: 0.05127271337130929.
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\basic.py:1487: UserWarning:

[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.007899104394606527. Current value: lambda_l2=0.007899104394606527


[I 2022-06-02 15:31:47,427] Trial 1 finished with value: 0.039180613212411834 and parameters: {'n_estimators': 3424, 'max_depth': 8, 'colsample_bytree': 0.6, 'lambda': 0.007899104394606527, 'alpha': 0.4699235098517489, 'subsample': 0.6}. Best is trial 1 with value: 0.039180613212411834.
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\basic.py:1487: UserWarning:

[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.002512232952810003. Current value: lambda_l2=0.002512232952810003


[I 2022-06-02 15:31:50,707] Trial 2 finished with value: 0.07390562028728377 and parameters: {'n_estimators': 3358, 'max_depth': 12, 'colsample_bytree': 0.8, 'lambda': 0.002512232952810003, 'alpha': 9.139877522567168, 'subsample': 1.0}. Best is trial 1 with value: 0.039180613212411834.
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\rihot\Anaconda3\envs\deep\lib\site-packages\lightgbm\basic.py:1487: UserWarning: 

[LightGBM] [Warning] Unknown parameter: predictor
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=1.8562768699973728. Current value: lambda_l2=1.8562768699973728


KeyboardInterrupt: 